In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
data=pd.read_csv('../raw_data/grade_8_reading.csv')
data.head()

,Year,Jurisdiction,Gender,Average scale score
0,2022,National,Male,256
1,2022,National,Female,265
2,2022,Alabama,Male,245
3,2022,Alabama,Female,257
4,2022,Alaska,Male,245


In [3]:
## data clean
data.replace('—', np.nan, inplace=True)
data.replace('‡', np.nan, inplace=True)
data['Year'] = pd.to_numeric(data['Year'], errors='coerce')
data.dropna(subset=['Year'], inplace=True)  # Drop rows where 'Year' is NaN
data['Year'] = data['Year'].astype(int)
data['Average scale score'] = pd.to_numeric(data['Average scale score'], errors='coerce')

In [4]:
# only 2000 years later
data = data[data['Year'] > 2000]
data = data[data['Jurisdiction'] != 'DoDEA']
data = data[data['Jurisdiction'] != 'Puerto Rico']
data = data[data['Jurisdiction'] != 'District of Columbia']

In [5]:
data['Year'] = data['Year'].astype(str)

In [6]:
null_exists = data['Average scale score'].isnull().sum()
null_exists

18

In [7]:
national= data[data['Jurisdiction'] == 'National']
national

,Year,Jurisdiction,Gender,Average scale score
0,2022,National,Male,256.0
1,2022,National,Female,265.0
106,2019,National,Male,258.0
107,2019,National,Female,269.0
212,2017,National,Male,262.0
213,2017,National,Female,272.0
318,2015,National,Male,261.0
319,2015,National,Female,270.0
424,2013,National,Male,263.0
425,2013,National,Female,273.0


In [8]:
# Calculate the national average for each year and gender
national_averages = data[data['Jurisdiction'] == 'National'].groupby(['Year', 'Gender'])['Average scale score'].mean().reset_index()
# Merge the national averages back into the original dataset on 'Year' and 'Gender'
data = pd.merge(data, national_averages, on=['Year', 'Gender'], how='left', suffixes=('', '_national'))
#data['Average scale score_national']=data['Average scale score_national'].astype(int)

In [9]:
# Fill null values in 'Average scale score' with the corresponding national average
data['Average scale score'].fillna(data['Average scale score_national'], inplace=True)

# Remove the auxiliary column used for filling nulls
data.drop(columns=['Average scale score_national'], inplace=True)

In [10]:
null_exists = data['Average scale score'].isnull().sum()
null_exists

0

In [13]:
data.to_csv('../clean_data/after_clean.csv',index=False)